In [ ]:
%set_env SM_FRAMEWORK=tf.keras

In [ ]:
import dataset

In [ ]:
import tensorflow as tf
import segmentation_models as sm

In [ ]:
model = sm.Unet('efficientnetb2', classes=1, activation='sigmoid')
model.load_weights('models/data_l_100e_b2.h5')

In [ ]:
data_p_dir = 'data_p/s/'
ds = dataset.train_dataset(data_p_dir + 'image/', data_p_dir + 'label/')

BATCH_SIZE = 2
item = ds.batch(BATCH_SIZE).skip(0).take(2)
for image, mask in item:
    pred = model.predict(image)
    for i in range(BATCH_SIZE):
        dataset.plot_image_mask_predict(image[i], mask[i], pred[i])
